In [1]:
import os
import time
import pandas as pd

# Constants
DATA_DIR = fr"E:\Space-Time-Prism-Model\PwoD STP\Data"
OUTPUT_DIR = fr"E:\Space-Time-Prism-Model\PwoD STP\Data"
MODES = ['Walk', 'Bike', 'Transit', 'Drive']
MODE_REPLACEMENTS = {'auto': 'Drive','walk': 'Walk','transit': 'Transit','bike': 'Bike'}

def load_data(file_name):
    return pd.read_csv(os.path.join(DATA_DIR, file_name))

def convert_hhm_to_minutes(hhm):
    """Convert HHM format to minutes past midnight."""
    hours, minutes = divmod(hhm, 100)
    return (hours * 60) + minutes

def process_data(df):
    df = df.copy()  # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    for col in ['depart_hhm', 'arrive_hhm']:
        df[f'{col}_minutes'] = df[col].apply(convert_hhm_to_minutes)
    df['Total_F'] = df['arrive_hhm_minutes'] - df['depart_hhm_minutes']
    df['Total_T'] = df['Total_F'] - 5
    df = df[df['mode_t'] != 'other']
    df.loc[:, 'mode_t'] = df['mode_t'].replace(MODE_REPLACEMENTS)  # Use .loc to avoid SettingWithCopyWarning
    return df

def save_data(df, output_dir, file_prefix):
    os.makedirs(output_dir, exist_ok=True)
    for mode in MODES:
        mode_df = df[df['mode_t'] == mode]
        output_file_name = f'{file_prefix}_{mode}.csv'
        output_file_path = os.path.join(output_dir, output_file_name)
        mode_df.to_csv(output_file_path, index=False)
        print(f'Data for {mode} mode saved to {output_file_path}')

def main():
    # Record the start time
    start_time = time.time()

    # Load, process, and save origin data
    data_pwod = load_data('TAZ_origin_PwoD_PSM2.csv')
    # data_pwod=data_pwod[data_pwod['CO_NAME']=='SALT LAKE']
    data_pwod = process_data(data_pwod)
    save_data(data_pwod, OUTPUT_DIR, 'origin_pwod')

    # Load, process, and save destination data
    data_pwod_dest = load_data('TAZ_destination_PwoD_PSM2.csv')
    # data_pwod_dest=data_pwod_dest[data_pwod_dest['CO_NAME']=='SALT LAKE']
    data_pwod_dest = process_data(data_pwod_dest)
    save_data(data_pwod_dest, OUTPUT_DIR, 'destn_pwod')

    # Record the end time and print the total processing time
    end_time = time.time()
    print(f"The script took {end_time - start_time} seconds to complete.")

if __name__ == "__main__": 
    main() 

Data for Walk mode saved to E:\Space-Time-Prism-Model\PwoD STP\Data\origin_pwod_Walk.csv
Data for Bike mode saved to E:\Space-Time-Prism-Model\PwoD STP\Data\origin_pwod_Bike.csv
Data for Transit mode saved to E:\Space-Time-Prism-Model\PwoD STP\Data\origin_pwod_Transit.csv
Data for Drive mode saved to E:\Space-Time-Prism-Model\PwoD STP\Data\origin_pwod_Drive.csv
Data for Walk mode saved to E:\Space-Time-Prism-Model\PwoD STP\Data\destn_pwod_Walk.csv
Data for Bike mode saved to E:\Space-Time-Prism-Model\PwoD STP\Data\destn_pwod_Bike.csv
Data for Transit mode saved to E:\Space-Time-Prism-Model\PwoD STP\Data\destn_pwod_Transit.csv
Data for Drive mode saved to E:\Space-Time-Prism-Model\PwoD STP\Data\destn_pwod_Drive.csv
The script took 0.14799785614013672 seconds to complete.
